In [60]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [61]:
# change path to chromedriver on your computer
PATH = "/home/gsunit/development/chromedriver"

url = "https://www.ubereats.com/city/new-york-city"

In [62]:
driver = webdriver.Chrome(PATH)
driver.get(url)
print("Successfully opened: ", driver.title)

Successfully opened:  Best New York City Restaurants | Food Delivery & Takeout


In [63]:
results = driver.find_elements_by_class_name("b2.fc")
len(results)

88

In [64]:
import pandas as pd

df = pd.DataFrame(columns=['name', 'offer', 'delivery_fee', 'delivery_time', 'rating', 'cuisine', 'url'])
# df = pd.DataFrame(columns=['rating', 'cuisine'])

In [65]:
i=0
for result in results:
#     print(i)
    lines = result.text.split('\n')
    n = len(lines)
    df.loc[i] = [
        lines[-8] if n>=8 else lines[0],
        lines[0] if n==9 else " ",
        lines[-6].split(" ")[0] if n>=6 else " ",
        lines[-4] if n>=4 else " ",
        lines[-2],
        lines[-1],
        result.find_element_by_tag_name("a").get_attribute('href')
    ]
    i = i+1

In [66]:
df.head()

,name,offer,delivery_fee,delivery_time,rating,cuisine,url
0,Wendy's (650 Broadway),,$0.99,30–45 Min,4.2,American • Burgers • Fast Food • Breakfast and...,https://www.ubereats.com/new-york/food-deliver...
1,Bluestone Lane (51 Astor Pl),,$1.99,35–50 Min,4.7,Breakfast and Brunch • Coffee and Tea • Health...,https://www.ubereats.com/new-york/food-deliver...
2,Murray's Bagels - Greenwich Village,,$3.99,40–55 Min,4.8,Breakfast and Brunch • Sandwich • Salads • Des...,https://www.ubereats.com/new-york/food-deliver...
3,La Parisienne - FiDi,,$0.99,20–35 Min,5.0,French • Cafe • Healthy,https://www.ubereats.com/new-york/food-deliver...
4,Red Mango - FiDi,,$1.99,30–45 Min,4.8,Desserts,https://www.ubereats.com/new-york/food-deliver...


In [67]:
from selenium.webdriver import ActionChains
from selenium.webdriver.common.keys import Keys

# df_t = df.head(3)
df['num_reviews'] = "( )"
df['address'] = ""

main_win_handle = driver.current_window_handle

for i in range(df.shape[0]):
    name = df.iloc[i]['name']
    print(f"Processing: {name}")
    url = driver.find_element_by_link_text(name)
    actions = ActionChains(driver)
    actions.key_down(Keys.CONTROL).click(url).key_up(Keys.CONTROL).perform()
    driver.implicitly_wait(4)
    driver.switch_to.window(driver.window_handles[-1])
    driver.implicitly_wait(4)
    try:
        num_reviews1 = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, 'do'))
        )
        num_reviews2 = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, 'dk'))
        )
    except:
        print(f"error1: {name}")
        
    driver.implicitly_wait(4)
    
    if len(num_reviews1.text)>0 and num_reviews1.text[0]=='(' :
        num_reviews = num_reviews1.text
    elif len(num_reviews2.text)>0 and num_reviews2.text[0]=='(' :
        num_reviews = num_reviews2.text
    else :
        num_reviews = '( )'
        
    print(num_reviews)
#     df.iloc[i]['num_reviews'] = final_num
#     df.iloc[i]['address'] = final_address
    
    driver.close()
    driver.switch_to_window(main_win_handle)
    driver.back()
    driver.implicitly_wait(4)
    

Processing: Wendy's (650 Broadway)
(271)
Processing: Bluestone Lane (51 Astor Pl)
(144)
Processing: Murray's Bagels - Greenwich Village
(55)
Processing: La Parisienne - FiDi
( )


/home/gsunit/my-env/lib/python3.6/site-packages/ipykernel/__main__.py:43: DeprecationWarning: use driver.switch_to.window instead


InvalidSessionIdException: Message: invalid session id


In [ ]:
df

In [ ]:
df_t